# **FFT**

This Notebook is for implementing the FFT algorithm for computing the DTFT in Python to understand its inner working.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import timeit
from scipy.fftpack import fft, fftshift, fftfreq

## **Generate some sample data to play with**

In [ ]:
fs = 1e3
T = 10
t_ax = np.arange(0, T, 1/fs)
f = 10
x = np.sin(2 * np.pi * t_ax)

## **Using standard libraries**

In [ ]:
# take the FFT
X = fft(x)
f_ax = fftfreq(len(x), 1/fs)
X = fftshift(X)
f_ax = fftshift(f_ax)

In [ ]:
# plot the PSD
# look at the frequency spectrum of the original and transformed signals
plt.figure(figsize=(10,5))
plt.plot(f_ax, np.abs(X)**2, c='b', lw=2)
plt.xlim(0,5)
plt.xlabel("freq / Hz")
plt.ylabel("$|X[f]|^{2}$")
plt.title("PSD")
plt.show()

## **From scratch**

#### **Function definitions**

In [ ]:
# function definitions
def dft(x):
    x = np.asarray(x, dtype=float)                                  # convert input to numpy array
    N = x.shape[0]                                                  # number of samples in signal (and DFT)
    n = np.arange(N)                                                # 'time-domain' axis
    k = n.reshape((N,1))                                            # 'frequency-domain' axis
    M = np.exp(-2j * np.pi * k * n / N)                             # matrix with values of n along one dim and k along the other
    return np.dot(M, x)                                             # return vector along k dimension

In [ ]:
# define using recursion
def fft_scratch(x):
    x = np.asarray(x, dtype=float)                                  # convert image to numpy array
    N = x.shape[0]                                                  # number of samples in signal (and DFT)
    if N % 2 > 0:
        raise ValueError("must be a power of 2")
    elif N <= 2:                                                    # only computes DFT once signal has been split into signals of length 2 
        return dft(x)
    else:
        X_even = fft(x[::2])                                        # keeps splitting signal into 'odd' and 'even' part recursively
        X_odd = fft(x[1::2])                                        # (recursion)
        terms = np.exp(-2j * np.pi * np.arange(N) / N)
        return np.concatenate([X_even + terms[:int(N/2)] * X_odd,
                               X_even + terms[int(N/2):] * X_odd])

In [ ]:
# define using vector operations
def fft_scratch_v(x):
    x = np.asarray(x, dtype=float)
    N = x.shape[0]
    if np.log2(N) % 1 > 0:
        raise ValueError("must be a power of 2")
        
    N_min = min(N, 2)
    
    n = np.arange(N_min)
    k = n[:, None]
    M = np.exp(-2j * np.pi * n * k / N_min)
    X = np.dot(M, x.reshape((N_min, -1)))
    while X.shape[0] < N:
        X_even = X[:, :int(X.shape[1] / 2)]
        X_odd = X[:, int(X.shape[1] / 2):]
        terms = np.exp(-1j * np.pi * np.arange(X.shape[0])
                        / X.shape[0])[:, None]
        X = np.vstack([X_even + terms * X_odd,
                       X_even - terms * X_odd])
    return X.ravel()

#### **Compute PSD via FFTs**

In [ ]:
X = fft_scratch(x)